## 1. 협업필터링

### 1-1. 협업필터링 개요

1) 정의 : 사용자의 구매 패턴이나 평점을 가지고 다른 사람들의 구매 패턴, 평점을 통해 추천

2) 종류 : 최근접 이웃 기반 / 잠재 요인 기반

3) Neighborhood based method(이웃 기반)
    - 정의 : 메모리 기반 알고리즘.
    - 알고리즘 : 
        - User-based collaborative filtering : 사용자의 구매패턴(평점)과 유사한 사용자를 찾아서 추천리스트 생성
        - Item-based collaborative filtering : 특정 사용자가 준 점수간의 유사한 상품을 찾아서 추천 리스트 생성
    - KNN(K Nearest Neighbors) : 가장 근접한 K명의 Neighbors를 통해서 예측하는 방법
        - 데이터(Explicit Feedback) : 유저가 자신의 선호도를 직접 표현한 데이터
    - 장점 :
        - 간단하고 직관적이다.
        - 특정 item을 추천하는 이유를 정당화하기 쉽다. item기반 방법의 해석 가능성이 두드러진다.
        - 추천 리스트에 새로운 item과 user가 추가되어도 상대적으로 안정적이다
    - 단점 : 
        - user 기반 방법의 시간, 속도, 메모리가 많이 필요
        - 희소성 때문에 제한된 범위가 있다
            - John의 top-K만 관심이 있다.
            - John과 비슷한 이웃 중 아무도 해리포터를 평가하지 않으면, 해리포터에 대한 예측을 제공할 수 없다

4) Latent Factor Collaborative Filtering(잠재 요인 기반)
    - 정의 : Rating Matrix에서 빈 공간을 채우기 위해 사용자와 상품을 잘 표현하는 차원(Latent Factor)을 찾는 방법.
    - 원리 : 사용자의 잠재요인과 아이템의 잠재요인을 내적해서 평점 매트릭스를 계산
        - Observed Only MF, Weighted MF, SVD
    - SGD
        - 정의 : 고유값 분해와 같은 행렬을 대각화 하는 방법
        - 진행 과정 :
            - User Latent와 Item Latent의 임의로 초기화
            - Gradient Descent 진행
            - 모든 평점에 대해서 반복(epoch = 1)
            - 2~3의 과정을 10회 반복(epoch = 10)
        - 장점 : 매우 유연한 모델로 다른 Loss Function을 사용할 수 있고, parallelized 가능
        - 단점 : 수렴까지 속도가 매우 느리나 딥러닝을 통해 해결 가능
    - ALS
        - 정의 : 기존의 SGD가 두개의 행렬을 동시에 최적화 한다면, ALS는 두 행렬 중 하나를 고정시키고 다른 하나의 행렬을 순차적으로 반복하면서 최적화
        - 알고리즘 :
            - 초기 아이템, 사용자 행렬을 초기화
            - 아이템 행렬을 고정하고 사용자 행렬을 최적화
            - 사용자 행렬을 고정하고 아이템 행렬을 최적화
            - 위의 2, 3 과정을 반복
            
5) 장점
    - 도메인 지식이 필요하지 않음
    - 사용자의 새롱누 흥미를 발견하기 좋음
    - 시작단계의 모델로 선택하기 좋음(추가적인 문맥 정보 등이 필요하지 않음)
    
6) 단점
    - 새로운 아이템에 대해서 다루기가 어려움
    - side feature(고객의 개인정보, 아이템의 추가정보)를 포함시키기 어려움
            

### SGD 실습

In [47]:
import numpy as np
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization() :
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose = False):
        self._R = R #Rating Matrix
        self._num_users, self._num_items = R.shape #사용자수, 아이템수
        self._k = k # Latent space 의 크기(설정해주면 됨)
        self._learning_rate = learning_rate #학습률
        self._reg_param = reg_param #weight의 regularization 값
        self._epochs = epochs #전체 학습 횟수
        self._verbose = verbose # 학습 과정을 출력할지 여부

In [38]:
def get_prediction(self, i, j):
    return self._b + self._b_P[i] + self._b_Q[i] + self._P[i, :].got(self._Q[j: ,].T)
    # User Latent space 와 Item latent space의 곱에 biases를 더함

In [39]:
def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2)
        return np.sqrt(cost) / len(xi)

In [40]:
def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq

In [41]:
def gradient_descent(self, i, j, rating):
    #error term 계산
    prediction = self.get_prediction(i, j)
    error = rating - prediction
    
    # biases 업데이트
    self._b_P[i] += self._learning_rate * (error - self._req_param * self._b_P[i])
    self._b_Q[i] += self._learning_rate * (error - self._req_param * self._b_Q[i])
    
    # latent feature 업데이트
    dp, dq = self.gradient(error, i, j)
    self._P[i, :] += self._learning_rate * dp
    self._Q[i, :] += self._learning_rate * dq

In [42]:
def fit(self) :
    #p와 q라는 latent space 생성
    self._P = np.random.normal(size = (self._num_users, self._k))
    self._Q = np.random.normal(size = (self._num_items, self._k))
    
    #biases term 설정
    self._b_P = np.zeros(self._num_users)
    self._b_Q = np.zeros(self._num_items)
    self._b = np.mean(self._R[np.where(self._R != 0)])
    
    # train
    self._training_process = []
    for epoch in range(self._epochs) :
        xi, yi = self._R.nonzero() # 평점이 0이 아닌 것만 진행
        for i, j in zip(xi, yi):
            self.gradient_descent(i, j, self._R[i, j])
        cost = self.cost()
        self._training_process.append((epoch, cost))
        
        if self._verbose == True and ((epoch+ 1) % 10 == 0):
            print("Iteration : %d ; cost = %.4f" % (epoch +1, cost))

In [49]:
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """

        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose


    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k))
        self._Q = np.random.normal(size=(self._num_items, self._k))

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)])

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):

            # rating이 존재하는 index를 기준으로 training
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0:
                        self.gradient_descent(i, j, self._R[i, j])
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2)
        return np.sqrt(cost) / len(xi)


    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)


    def print_results(self):
        """
        print fit results
        """

        print("User Latent P:")
        print(self._P)
        print("Item Latent Q:")
        print(self._Q.T)
        print("P x Q:")
        print(self._P.dot(self._Q.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_P)
        print("Item Latent bias:")
        print(self._b_Q)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])

In [50]:
if __name__ == "__main__":
    # rating matrix - User X Item : (7 X 5)
    R = np.array([
        [1, 0, 0, 1, 3],
        [2, 0, 3, 1, 1],
        [1, 2, 0, 5, 0],
        [1, 0, 0, 4, 4],
        [2, 1, 5, 4, 0],
        [5, 1, 5, 4, 0],
        [0, 0, 0, 1, 0],
    ])

    # P, Q is (7 X k), (k X 5) matrix
    # 데이터 값이 작아 k = 3을 썼지만 보통 인자값으로는 20을 많이 사용한다.
    factorizer = MatrixFactorization(R, k=3, learning_rate=0.01, reg_param=0.01, epochs=300, verbose=True)
    factorizer.fit()
    factorizer.print_results()

Iteration: 10 ; cost = 0.2325
Iteration: 20 ; cost = 0.1616
Iteration: 30 ; cost = 0.1236
Iteration: 40 ; cost = 0.1000
Iteration: 50 ; cost = 0.0830
Iteration: 60 ; cost = 0.0698
Iteration: 70 ; cost = 0.0591
Iteration: 80 ; cost = 0.0503
Iteration: 90 ; cost = 0.0430
Iteration: 100 ; cost = 0.0369
Iteration: 110 ; cost = 0.0318
Iteration: 120 ; cost = 0.0275
Iteration: 130 ; cost = 0.0239
Iteration: 140 ; cost = 0.0208
Iteration: 150 ; cost = 0.0182
Iteration: 160 ; cost = 0.0160
Iteration: 170 ; cost = 0.0142
Iteration: 180 ; cost = 0.0126
Iteration: 190 ; cost = 0.0112
Iteration: 200 ; cost = 0.0101
Iteration: 210 ; cost = 0.0091
Iteration: 220 ; cost = 0.0083
Iteration: 230 ; cost = 0.0076
Iteration: 240 ; cost = 0.0070
Iteration: 250 ; cost = 0.0065
Iteration: 260 ; cost = 0.0060
Iteration: 270 ; cost = 0.0056
Iteration: 280 ; cost = 0.0053
Iteration: 290 ; cost = 0.0050
Iteration: 300 ; cost = 0.0048
User Latent P:
[[ 0.24752074  0.29420478 -1.54026316]
 [-0.99481879 -0.3696489 